# Tutorial #2: How to query data from BigQuery in a Notebook

Welcome to part II of our tutorials designed for participants in the IronHacks. In this second notebook (part II), we will show you how you can access our training data stored in BigQuery using a key stored in your user profile.

> Before you get started: This tutorial will not more 10 min and you should be able to work with our training data right after. 

**Our goal**: Help you getting started with the R packages `BigRQuery` or `DBI` or `dply` in order to access a training dataset stored in BigQuery! So if you have never used these libraries before this tutorial is key for you. 

**BigQuery - What's that?**: 
BigQuery is Google's flagship data warehousing system: "Serverless, highly scalable, and cost-effective multi-cloud data warehouse designed for business agility". It allows you to analyze large amount of data using ANSI SQL at blazing-fast speeds, with zero operational overhead. You can find out more [here](https://cloud.google.com/bigquery). 


**Why do we use BigQuery?**
In the COVID-19 Data Science Challenges you will use BIG DATA from our data providers SafeGraph, the Management Performance Hub (MPH), and other partners (Department of Workforce Development). The first hack Summer 2020, will use preprocessed data so you will not need to use all the functionalities of BIG QUERY as we have sampled down more than 50 datasets with more than 1 TB and millions of raws into a small sets of cleaned tables without missing entries and clear identifiers. However, using BigQuery will still be very helpful as you can see for exploring data without having to use them in memory etc. It will also set you up for the future of data science since BigQuery is replacing other BIG DATA services (e.g. Spark). 

**How do we give you access to BigQuery?**
In Big Query data are stored in projects. Inside a project there are multiple datasets. Each dataset can contain multiple tables. 
In this hack we give you access to a project called: `ironhacks-covid19-data`. In this project there are two datasets:`ironhacks-covid19-data:ironhacks_covid19_training` and `ironhacks-covid19-data:ironhacks_covid19_competition
`. During the training period you will only find data in the first dataset. In this first tutorial we only use one first relatively simply structured table stored in this dataset. It is called `covid19_tests_cases_deaths_IN`

> Keep in mind: In this tutorial you will learn how to get access to the `ironhacks-covid19-data` and the datset `ironhacks-covid19-data:ironhacks_covid19_training` stored inside this project.

**What's BigRQuery**: 
`BigRQuery` package allows you to query data stored in BigQuery You can find the official documentation [here}(https://bigrquery.r-dbi.org/)

**What's DBS**
The `DBI` package helps connecting R to database management systems (DBMS). `DBI` separates the connectivity to the DBMS into a “front-end” and a “back-end”. It defines an interface that is implemented by the "backend". In our case the backend is BigRQuery (but it also uses other backends such as SQLlite etc.). You can find the official documentation [here](https://dbi.r-dbi.org/) 

**What's dplry**
`dplry` is is a grammar of data manipulation, providing a consistent set of verbs that help you solve the most common data manipulation challenges. It is part of the `tidyverse` package. You can find the official documentation [here](https://dplyr.tidyverse.org/) The `dplyr` interface lets you treat BigQuery tables as if they are in-memory data frames. This is the most convenient layer if you don’t want to write SQL, but instead want dbplyr to write it for you.

> the following tutorial is structured in four section! So let's get started

## Section I: Loading the libraries

In a first step, we will be loading the libraries we need. I load the complete tidyverse package in case I want to also plot later but you can also just load dplyr

In [182]:
lapply (c("bigrquery","tidyverse","DBI"),require, character.only=TRUE) ###loading the libraries I need

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

> As mentioned in part 1 of our tutorial, there is no installation required for you. So this should do the job very fast for you. 

## Section II: Authorizing your BigQuery Access

1. Getting your key
2. Setting up the autorization
3. And finding out more what you can do with this key!

So the next step now is to find the keys: 1) Go to your user profile, 2) click on *Download your hack dataset training key* and 3) Upload it to your Juptyer lab environment. Below you find a screenshot where you can find your key. 

![image](https://raw.githubusercontent.com/ironhacks/Tutorials-COVID-19/master/part-2/profile_image.jpg?token=AC7DAY5X2TFRU6J6IU7ZQ5S7FXKP2)

> After you have added the key to your file structure in your lab environment you can authorize your access! 

In [186]:
bq_auth(path = "/home/jovyan/Tutorials/service-account.json") #authorizing the token! 

> **This is an important step***: please keep in mind that you need to put the path `/home/jovyan/` followed by your directory name and the name that you give to your json file! 

In [187]:
bq_has_token() #confirms that bq has token

[1] TRUE

> **Yeah it is working!**

In [188]:
bq_projects(page_size = 10, max_pages = 1, warn = TRUE) # now I am getting the projects that are associated with that token! Yeah - it is the right one.

[1] "ironhacks-covid19-data"

In [189]:
bq_project_datasets('ironhacks-covid19-data', page_size = 10, max_pages = 1, warn = TRUE) # now you also see the datasets in the project 

[[1]]
<bq_dataset> ironhacks-covid19-data.ironhacks_covid19_competition

[[2]]
<bq_dataset> ironhacks-covid19-data.ironhacks_covid19_training


**Next** we are establishing the database connection! 

In [24]:
projectid<-'ironhacks-covid19-data'
datasetid<-'ironhacks_covid19_training'
bq_conn <-  DBI::dbConnect(bigquery(), 
                            project = projectid,
                            dataset = datasetid,
                            use_legacy_sql = FALSE
                      ) ## setting up the project and the database connection

In [25]:
DBI::dbListTables(bq_conn) ##checking the tables in there! 

[1] "covid19_tests_cases_deaths_IN"

> Yeah! Now we have what we need to move on! We have also prepared a sheet with the schema of the database so that it is easier for you to understand what is in there. It can be found [here](https://docs.google.com/spreadsheets/d/1s591Ajha7KJMM0syot0Nc3fRRgEaT8zIU4BCPKeyEzk/edit#gid=557548514). 

## Section III: Loading table and understanding it
So what we do: 
* Using dplyr to load the table
* Getting the column names etc. 
* Transforming it into a dataframe and understanding it further (number of rows, columns, DATE range, missing entries)
* Using DBI library to send a query and fetch the table

The table `covid19_tests_cases_deaths_IN` can be easily accessed by using the dplyr::tbl() function and passing the table name and BigQuery connection details as parameters. We will do this next

In [33]:
table<-dplyr::tbl(bq_conn,'covid19_tests_cases_deaths_IN')

Notice that the “table” is of class tbl_sql and not a tibble. This is because tbl() creates a direct reference to the table in BigQuery but does not bring the data in-memory.

In [ ]:
class(table)

In [32]:
colnames(table)

[1] "DATE"                "COVID_TEST"          "DAILY_DELTA_TESTS"  
 [4] "DAILY_BASE_TESTS"    "COVID_DEATHS"        "DAILY_DELTA_DEATHS" 
 [7] "DAILY_BASE_DEATHS"   "COVID_COUNT"         "DAILY_DELTA_CASES"  
[10] "DAILY_BASE_CASES"    "COVID_COUNT_CUMSUM"  "COVID_DEATHS_CUMSUM"
[13] "COVID_TEST_CUMSUM"

In [191]:
nrow(as.data.frame(table))
length(as.data.frame(table))
dim(as.data.frame(table))

[1] 152

[1] 13

[1] 152  13

In [51]:
head(table)

DATE,COVID_TEST,DAILY_DELTA_TESTS,DAILY_BASE_TESTS,COVID_DEATHS,DAILY_DELTA_DEATHS,DAILY_BASE_DEATHS,COVID_COUNT,DAILY_DELTA_CASES,DAILY_BASE_CASES,COVID_COUNT_CUMSUM,COVID_DEATHS_CUMSUM,COVID_TEST_CUMSUM
<date>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
2020-02-26,1,0,1,0,0,0,0,0,0,0,0,1
2020-02-27,1,0,1,0,0,0,0,0,0,0,0,2
2020-02-29,1,0,1,0,0,0,0,0,0,0,0,3
2020-03-02,2,0,2,0,0,0,0,0,0,0,0,5
2020-03-03,4,0,4,0,0,0,0,0,0,0,0,9
2020-03-04,4,0,4,0,0,0,0,0,0,0,0,13


In [133]:
range(table$DATE) #what's the range of the time series

[1] "2020-02-26" "2020-07-28"

In [132]:
sum(is.na(table)) #are there missing entries? 

[1] 0

> Great work! So now you could get started with actually working with the data, and manipulating it (e.b. subsetting etc.) but before we do that I want to make sure you also learn how to do with this the `DBI` library and the `dbSendQuery` and the `dbFetch` command! 

In [147]:
rs<-dbSendQuery(bq_conn,"SELECT * FROM covid19_tests_cases_deaths_IN")
head(dbFetch(rs))

DATE,COVID_TEST,DAILY_DELTA_TESTS,DAILY_BASE_TESTS,COVID_DEATHS,DAILY_DELTA_DEATHS,DAILY_BASE_DEATHS,COVID_COUNT,DAILY_DELTA_CASES,DAILY_BASE_CASES,COVID_COUNT_CUMSUM,COVID_DEATHS_CUMSUM,COVID_TEST_CUMSUM
<date>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
2020-02-26,1,0,1,0,0,0,0,0,0,0,0,1
2020-02-27,1,0,1,0,0,0,0,0,0,0,0,2
2020-02-29,1,0,1,0,0,0,0,0,0,0,0,3
2020-03-02,2,0,2,0,0,0,0,0,0,0,0,5
2020-03-03,4,0,4,0,0,0,0,0,0,0,0,9
2020-03-04,4,0,4,0,0,0,0,0,0,0,0,13


In [163]:
class(rs)

[1] "BigQueryResult"
attr(,"package")
[1] "bigrquery"

Notice that "rs” is of class 'BigQueryResult' and not a tibble. This is because dbSendQuery() and dbFetch() create a direct reference to the table in BigQuery but does NOT bring the data in-memory. This is particularly useful if you work with big data! 

In [170]:
rs2<-dbReadTable(bq_conn,"covid19_tests_cases_deaths_IN")

In [175]:
head(rs2)

DATE,COVID_TEST,DAILY_DELTA_TESTS,DAILY_BASE_TESTS,COVID_DEATHS,DAILY_DELTA_DEATHS,DAILY_BASE_DEATHS,COVID_COUNT,DAILY_DELTA_CASES,DAILY_BASE_CASES,COVID_COUNT_CUMSUM,COVID_DEATHS_CUMSUM,COVID_TEST_CUMSUM
<date>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
2020-02-26,1,0,1,0,0,0,0,0,0,0,0,1
2020-02-27,1,0,1,0,0,0,0,0,0,0,0,2
2020-02-29,1,0,1,0,0,0,0,0,0,0,0,3
2020-03-02,2,0,2,0,0,0,0,0,0,0,0,5
2020-03-03,4,0,4,0,0,0,0,0,0,0,0,9
2020-03-04,4,0,4,0,0,0,0,0,0,0,0,13


[1] 13

In [180]:
nrow(rs2)
length(rs2)
dim(rs2)
sum(is.na(rs2))

[1] 152

[1] 13

[1] 152  13

[1] 0

## Section IV: Manipulating/querying the table
So what we will do: 
* Subsetting the table with dplr for DATE,COVID_TEST, COVID_COUNT and filtering for period 2020-04-01 to 2020-05-01
* Calculating the mean cases for the period
* Writing a sql query and executing it with the bigRquery bq_project_query() function and the bq_table_download() function


We can easily query the table in BigQuery using the dplr() functions select() and filter(), so we will do this now!

In [195]:
table2<-table %>% select(DATE,COVID_TEST,COVID_COUNT) %>% filter(DATE >="2020-04-01" & DATE <="2020-05-01")

In [196]:
head(table2)
range(table2$DATE)

DATE,COVID_TEST,COVID_COUNT
<date>,<int>,<int>
2020-04-11,1542,486
2020-04-01,2543,468
2020-04-02,2701,396
2020-04-04,1636,461
2020-04-06,2425,556
2020-04-10,2727,514


[1] "2020-04-01" "2020-05-01"

In [197]:
nrow(table2)

[1] 31

In [198]:
mean(table2$COVID_TEST)

[1] 3243.548

> Great. We got this sorted. Now, I wanted to also show you how to do the same thing by writing an sql query and using the `bq_project_query()` and `bq_table_download()` functionality (Note, we could also do the `DBI::dbSendQuery()` method that I described above but as a final step I am going to use `bq_project_query()`

In [106]:
query <-"SELECT DATE, COVID_TEST, COVID_COUNT FROM ironhacks_covid19_training.covid19_tests_cases_deaths_IN WHERE (DATE BETWEEN '2020-04-01' AND '2020-05-01')
ORDER BY DATE"

In [107]:
tb2<-bq_project_query(projectid,query)

In [108]:
bq_table_download(tb2)

DATE,COVID_TEST,COVID_COUNT
<date>,<int>,<int>
2020-04-01,2543,468
2020-04-02,2701,396
2020-04-03,2802,513
2020-04-04,1636,461
2020-04-05,1095,527
2020-04-06,2425,556
2020-04-07,2782,422
2020-04-08,3073,422
2020-04-09,2749,558


In [127]:
query2 <-"SELECT AVG(COVID_TEST) FROM (SELECT DATE, COVID_TEST, COVID_COUNT FROM ironhacks_covid19_training.covid19_tests_cases_deaths_IN WHERE (DATE BETWEEN '2020-04-01' AND '2020-05-01')
ORDER BY DATE)"

In [128]:
tb3<-bq_project_query(projectid,query2)

In [129]:
tb3

<bq_table> ironhacks-covid19-data._aa1a5f527e915a9f592ea8cd7cf170727d4c67ed.anon043318ec962478bc82ac41b5b32693654073cbd4

In [130]:
class(tb3)

[1] "bq_table"

In [131]:
bq_table_download(tb3)

f0_
<dbl>
3243.548


> **THIS IS IS IT FOR NOW!** So now it is time to practice using the method you like most! Please put our questions into the forum!